## Scraping HTML Site for Music Data

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time
import csv
import numpy as np

In [2]:
# Load the archival site into a request and into the web scraper
url = "https://kworb.net/ww/archive/"

response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
# Use Beautiful Soup to find all the anchor tags within the "pre" tag of the HTML content
pre_tag = soup.find('pre')
a_tags = pre_tag.find_all('a')

In [4]:
# Loop through the list of links and use requests to make a request to each link but only that start between 2015-2021
links_2015 = []
links_2016 = []
links_2017 = []
links_2018 = []
links_2019 = []
links_2020 = []
links_2021 = []

for a in a_tags:
    if a.has_attr('href') and a['href'].startswith(('2015')):
        links_2015.append(a['href'])
    if a.has_attr('href') and a['href'].startswith(('2016')):
       links_2016.append(a['href'])
    if a.has_attr('href') and a['href'].startswith(('2017')):
       links_2017.append(a['href'])
    if a.has_attr('href') and a['href'].startswith(('2018')):
       links_2018.append(a['href'])
    if a.has_attr('href') and a['href'].startswith(('2019')):
       links_2019.append(a['href'])
    if a.has_attr('href') and a['href'].startswith(('2020')):
       links_2020.append(a['href'])
    if a.has_attr('href') and a['href'].startswith(('2021')):
       links_2021.append(a['href'])

##### Testing HTML Anatomy

In [6]:
print(links_2015)

['20150101.html', '20150102.html', '20150103.html', '20150104.html', '20150105.html', '20150106.html', '20150107.html', '20150108.html', '20150109.html', '20150110.html', '20150111.html', '20150112.html', '20150113.html', '20150114.html', '20150115.html', '20150116.html', '20150117.html', '20150118.html', '20150119.html', '20150120.html', '20150121.html', '20150122.html', '20150123.html', '20150124.html', '20150125.html', '20150126.html', '20150127.html', '20150128.html', '20150129.html', '20150130.html', '20150131.html', '20150201.html', '20150202.html', '20150203.html', '20150204.html', '20150205.html', '20150206.html', '20150207.html', '20150208.html', '20150209.html', '20150210.html', '20150211.html', '20150212.html', '20150213.html', '20150214.html', '20150215.html', '20150216.html', '20150217.html', '20150218.html', '20150219.html', '20150220.html', '20150221.html', '20150222.html', '20150223.html', '20150224.html', '20150225.html', '20150226.html', '20150227.html', '20150228.htm

In [18]:
# Verifying the url composition
url_test = str(links_2015[0])
print(url_test)

20150101.html


In [19]:
# Step 1: Use Beautiful Soup to parse the HTML content of each link and find the table tag within the HTML content.
link_test = requests.get(url+url_test)
link_soup_test = BeautifulSoup(link_test.content, 'html.parser')

# Step 2: Find the table in the html code
table_test = link_soup_test.find('table')

# Step 3: Use pandas library to read the html table into dataframe
df_test = pd.read_html(str(table_test))[0].iloc[:,:9]

# Step 4: Extract the date code from the url link as use it as a week label for future sorting
weeklabel = url_test[:8]
df_test['Week'] = weeklabel

In [20]:
df_test.head()

,Pos,Pos+,Artist and Title,Days,Peak,Pts,Pts+,TPts,US,Week
0,1,=,Taylor Swift - Blank Space,67,1(29),20632,-115,1.113,2.0,20150101
1,2,=,Mark Ronson - Uptown Funk,52,2,19255,49,0.729,1.0,20150101
2,3,+1,Ed Sheeran - Thinking Out Loud,120,2,18928,464,1.444,6.0,20150101
3,4,-1,Taylor Swift - Shake It Off,136,1(47),18626,34,2.606,5.0,20150101
4,5,=,Meghan Trainor - All About That Bass,164,1(22),16812,70,2.539,7.0,20150101


#### Extract Code

In [5]:
# Define the lists of links
links = [links_2015, links_2016, links_2017, links_2018, links_2019, links_2020, links_2021]
week_allcharts_df = []

In [6]:
# Loop through each list of links
for link_list in tqdm(links, desc='Overall Year progress', position=0):
    # Loop through each link in the list
    for link in tqdm(link_list, desc=f'Links in {link_list[0][:4]}', position=1, leave=False):
        try:
        # Step 1: Use Beautiful Soup to parse the HTML content of each link and find the table tag within the HTML content.
            link_final = requests.get(url+link)
            link_soup_final = BeautifulSoup(link_final.content, 'html.parser')

        # Step 2: Find the table in the html code
            table_final = link_soup_final.find('table')

        # Step 3: Use pandas library to read the html table into dataframe
            weekchart_df = pd.read_html(str(table_final))[0].iloc[:,:9]

        # Step 4: Extract the date code from the url link as use it as a week label for future sorting
            weeklabel = link[:8]
            weekchart_df ['Week'] = weeklabel

        # Step 5: Append the dataframe to the list of dataframes
            week_allcharts_df.append(weekchart_df)
        except:
            print(f'Error processing link: {link}')

Overall Year progress: 100%|██████████| 7/7 [1:13:38<00:00, 631.27s/it]


In [8]:
weekchart_df.head()

,Pos,P+,Artist and Title,Days,Pk,(x?),Pts,Pts+,TPts,Week
0,1,+1,Elton John & Dua Lipa - Cold Heart,141,1,(x19),18842,890,2.522,20211231
1,2,-1,Adele - Easy On Me,78,1,(x49),18726,332,1.561,20211231
2,3,+13,Jaymes Young - Happiest Year,14,3,(x1),15922,8216,0.094,20211231
3,4,=,Acraze - Do It To It,78,4,NaN,14353,822,0.783,20211231
4,5,-2,Alesso & Katy Perry - When I'm Gone,3,3,(x1),13608,-1009,0.040,20211231


In [10]:
# Concatenate all dataframes in week_allcharts_df list into a single dataframe
week_allcharts_df = pd.concat(week_allcharts_df, ignore_index=True)

In [22]:
week_allcharts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565600 entries, 0 to 565599
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Pos               565600 non-null  int64  
 1   Pos+              133000 non-null  object 
 2   Artist and Title  565600 non-null  object 
 3   Days              565600 non-null  object 
 4   Peak              133000 non-null  object 
 5   Pts               565600 non-null  int64  
 6   Pts+              565600 non-null  int64  
 7   TPts              565600 non-null  float64
 8   US                65273 non-null   float64
 9   Week              565600 non-null  object 
 10  P+                432600 non-null  object 
 11  Pk                432600 non-null  float64
 12  (x?)              91685 non-null   object 
dtypes: float64(3), int64(3), object(7)
memory usage: 56.1+ MB


In [30]:
# export the dataframe to a CSV file with high efficiency
chunk_size = 10000 # adjust this to a size that works for your system
df_list = [chunk for _, chunk in week_allcharts_df.groupby(np.arange(len(week_allcharts_df)) // chunk_size)]

with open('../test chamber/week_allcharts_df_v01.csv', 'w') as f:
    writer = csv.writer(f)
    pbar = tqdm(total=len(week_allcharts_df))
    for chunk in df_list:
        chunk.to_csv(f, index=False, header=False)
        pbar.update(len(chunk))
    pbar.close()

100%|██████████| 565600/565600 [00:04<00:00, 122786.45it/s]
